#### Dashboard for investments

Idea: We want to invest when there is significant deviation away from the long-term mean

Steps:
1. Calculate long term trend of SNP500 - Loop through each time period in the dataframe and then conduct a ols regression and store the outputs. The bins are 15 year periods (12 * 15)
2. Determine at which points was there significant deviation away from trend (Jacksobian co-integration test)
3. Create threshold at which point there was significant deviation away that led you to an investment decision 
4. Create algorithm that invests when this happens

Data: 
1. Lets limit it to > 1900
2. Currently we just have a randomly CSV of the data, our goal is to find a constant datasource.

In [1]:
import os
import pandas as pd 
import numpy as np 
import matplotlib as plt 
from datetime import datetime

In [2]:
path = os.getcwd()
path

'/Users/dylanjohnson/Documents/bos_investments/bos_investments'

In [62]:
# env = os.chdir("./Documents/bos_investments")

In [345]:
df = pd.read_csv("s&p500.csv", infer_datetime_format = True)
df.head(6)

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
0,1871-01-01,4.44,0.26,0.4,12.46,5.32,89.00,5.21,8.02,NaN
1,1871-02-01,4.50,0.26,0.4,12.84,5.32,87.53,5.06,7.78,NaN
2,1871-03-01,4.61,0.26,0.4,13.03,5.33,88.36,4.98,7.67,NaN
3,1871-04-01,4.74,0.26,0.4,12.56,5.33,94.29,5.17,7.96,NaN
4,1871-05-01,4.86,0.26,0.4,12.27,5.33,98.93,5.29,8.14,NaN
5,1871-06-01,4.82,0.26,0.4,12.08,5.34,99.66,5.38,8.27,NaN


### Step 1: Create the bins

What do we want to do:
1. Create a bin for each time_stamp

In [346]:
df['Date'] = pd.to_datetime(df['Date'])

In [347]:
df = df[df['Date'] >= "1900-01-01"]

In [348]:
df

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
348,1900-01-01,6.10,0.22,0.48,7.90,3.15,192.98,6.88,15.19,18.67
349,1900-01-02,6.21,0.23,0.48,7.99,3.15,194.12,7.03,15.00,18.70
350,1900-01-03,6.26,0.23,0.48,7.99,3.14,195.69,7.27,15.00,18.78
351,1900-01-04,6.34,0.24,0.48,7.99,3.14,198.19,7.50,15.00,18.94
352,1900-01-05,6.04,0.25,0.48,7.80,3.13,193.41,7.93,15.37,18.40
...,...,...,...,...,...,...,...,...,...,...
1763,2017-01-12,2664.34,48.93,109.88,246.52,2.40,2700.13,49.59,111.36,32.09
1764,2018-01-01,2789.80,49.29,NaN,247.87,2.58,2811.96,49.68,NaN,33.31
1765,2018-01-02,2705.16,49.64,NaN,248.99,2.86,2714.34,49.81,NaN,32.12
1766,2018-01-03,2702.77,50.00,NaN,249.55,2.84,2705.82,50.06,NaN,31.99


In [349]:
a = pd.to_datetime(df['Date'])

In [350]:
a = np.array(a)

In [351]:
a

array(['1900-01-01T00:00:00.000000000', '1900-01-02T00:00:00.000000000',
       '1900-01-03T00:00:00.000000000', ...,
       '2018-01-02T00:00:00.000000000', '2018-01-03T00:00:00.000000000',
       '2018-01-04T00:00:00.000000000'], dtype='datetime64[ns]')

In [352]:
sp_500 = df["Real Price"]

In [353]:
sp_500

348      192.98
349      194.12
350      195.69
351      198.19
352      193.41
         ...   
1763    2700.13
1764    2811.96
1765    2714.34
1766    2705.82
1767    2642.19
Name: Real Price, Length: 1420, dtype: float64

Make a new column that tells the bins that it has

In [354]:
print(len(a))
rows = len(a)
print(rows)
bin_size = 15*12
print(bin_size)

1420
1420
180


In [355]:
col_names = []
for i in range(len(a)):
    col_names.append("bin_"+str(i)) 

In [356]:
bins = pd.DataFrame(columns = col_names)
bins = bins.reindex(list(range(0, rows))).reset_index(drop=True)
bins

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,...,bin_1410,bin_1411,bin_1412,bin_1413,bin_1414,bin_1415,bin_1416,bin_1417,bin_1418,bin_1419
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [357]:
n = rows
lists = [[] for _ in range(n)]
lists
bins.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'bin_5', 'bin_6', 'bin_7',
       'bin_8', 'bin_9',
       ...
       'bin_1410', 'bin_1411', 'bin_1412', 'bin_1413', 'bin_1414', 'bin_1415',
       'bin_1416', 'bin_1417', 'bin_1418', 'bin_1419'],
      dtype='object', length=1420)

In [358]:
columns = len(bins.columns)

In [359]:
print(len(bins))
print(len(bins.columns))
print(bin_size)

1420
1420
180


In [370]:
def create_empty_lists(rows):    
    n = rows
    lists = [[] for i in range(n)]
    list_cagr = []
    return lists, list_cagr
lists, list_cagr = create_empty_lists(rows)

In [337]:
#lists

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [361]:
## Note for some reason the number of rows = 1770 and the number of columns = 1688
def bin_defined(columns, rows, bin_size, empty_list, sp_500):
    for i in range(len(columns)): ## For each bin (column)
        for y in range(rows): #for each row
            if i <= y <= i+bin_size: ##for each index within our bin our length assign the index to the correct bin 
                lists[i].append(1)
            else:
                lists[i].append(0)       
    
    return lists
l_of_l = bin_defined(columns = bins.columns, rows = len(bins), bin_size = 180, empty_list = lists, sp_500 = sp_500)

In [371]:
l_of_l[0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [362]:
len(l_of_l)

1420

In [363]:
sp_500

348      192.98
349      194.12
350      195.69
351      198.19
352      193.41
         ...   
1763    2700.13
1764    2811.96
1765    2714.34
1766    2705.82
1767    2642.19
Name: Real Price, Length: 1420, dtype: float64

In [366]:
def convert_dataframe(l_of_l, sp_500,rows):
    
    m = np.array(l_of_l).T
    c = np.array(sp_500)
    np_bin = m * c[:, np.newaxis]
    
    col_names = []
    
    df = pd.DataFrame(np_bin.T)
    df = df.transpose()

    
    for i in range(len(l_of_l)):
        col_names.append("bin_" + str(i))
    df.columns = [col_names]
    
    df['date']= pd.date_range("1899-12-15", periods = rows, freq = "MS") ## Don't hardcode the starting date
    return df, np_bin 
df, np_bin = convert_dataframe(l_of_l, sp_500, rows)

In [367]:
df

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,...,bin_1411,bin_1412,bin_1413,bin_1414,bin_1415,bin_1416,bin_1417,bin_1418,bin_1419,date
0,192.98,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900-01-01
1,194.12,194.12,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900-02-01
2,195.69,195.69,195.69,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900-03-01
3,198.19,198.19,198.19,198.19,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900-04-01
4,193.41,193.41,193.41,193.41,193.41,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1900-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,2700.13,2700.13,2700.13,2700.13,2700.13,0.00,0.00,0.00,0.00,2017-12-01
1416,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,2811.96,2811.96,2811.96,2811.96,2811.96,2811.96,0.00,0.00,0.00,2018-01-01
1417,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,2714.34,2714.34,2714.34,2714.34,2714.34,2714.34,2714.34,0.00,0.00,2018-02-01
1418,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,2705.82,2705.82,2705.82,2705.82,2705.82,2705.82,2705.82,2705.82,0.00,2018-03-01


In [372]:
for i in rows(a):
    list_cagr.append()

IndentationError: expected an indented block (167399765.py, line 1)

In [379]:
bin_size

180

185.03

Make a column that gives the cagr for each bin size.
Steps: 
1. Get Highest and lowest from each column
2. Get the time difference

In [318]:
df['cagr'] = 

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
348,1900-01-01,6.10,0.22,0.48,7.90,3.15,192.98,6.88,15.19,18.67
349,1900-01-02,6.21,0.23,0.48,7.99,3.15,194.12,7.03,15.00,18.70
350,1900-01-03,6.26,0.23,0.48,7.99,3.14,195.69,7.27,15.00,18.78
351,1900-01-04,6.34,0.24,0.48,7.99,3.14,198.19,7.50,15.00,18.94
352,1900-01-05,6.04,0.25,0.48,7.80,3.13,193.41,7.93,15.37,18.40
...,...,...,...,...,...,...,...,...,...,...
1763,2017-01-12,2664.34,48.93,109.88,246.52,2.40,2700.13,49.59,111.36,32.09
1764,2018-01-01,2789.80,49.29,NaN,247.87,2.58,2811.96,49.68,NaN,33.31
1765,2018-01-02,2705.16,49.64,NaN,248.99,2.86,2714.34,49.81,NaN,32.12
1766,2018-01-03,2702.77,50.00,NaN,249.55,2.84,2705.82,50.06,NaN,31.99


In [ ]:
for i in rows

### Time for analysis 

####  What Is the Compound Annual Growth Rate (CAGR)?

The compound annual growth rate (CAGR) is the rate of return (RoR) that would be required for an investment to grow from its beginning balance to its ending balance, assuming the profits were reinvested at the end of each period of the investment’s life span. 

CAGR = (Xn/X0)^(1/t) - 1

where: 
- Xn = current market value of portfolio 
- X0 = initial portfolio's value 
- t = number of years

In [382]:
print(np_bin[:,0][0])
print(np_bin[:,0][bin_size])

192.98
185.03


In [414]:
bin_size

180

In [413]:
max(x_0)

1419

In [416]:
t = bin_size/12
x_0 = []
x_n = []
for i in range(len(a)):
    x_0.append(i)
    if i + bin_size <= len(a-1):
        x_n.append(i + bin_size) 
    else: 
        x_n.append(len(a - 1))

AttributeError: 'list' object has no attribute 'max'

In [435]:
sp_500_ls = [sp_500]

In [431]:
cagr = []
for i in range(len(a - 1)): 
    cagr.append(round(((sp_500[x_n[i]] / sp_500[x_0[i]]) ** (1/t) - 1) * 100 , 3))

KeyError: 180

In [433]:
x_n[1]

181

In [436]:
sp_500_ls[181]

IndexError: list index out of range

In [439]:
sp_ls = []
for i in sp_500:
    sp_ls.append(i)

In [441]:
sp_ls[181]

184.38

In [444]:
x_n[1]

181

In [445]:
T = [sp_ls[i] for i in x_n]

IndexError: list index out of range

In [443]:
sp_500[(x_n[1])]

KeyError: 181

In [426]:
bin_size - 1240

-1060

In [427]:
len(a)

1420

In [428]:
1420 - 1240

180

In [429]:
for i in range(len(a - 1)):
    print(i + np_bin[:, i-1])

[   0.      0.      0.   ...    0.      0.   2642.19]
[193.98 195.12 196.69 ...   1.     1.     1.  ]
[  2.   196.12 197.69 ...   2.     2.     2.  ]
[  3.     3.   198.69 ...   3.     3.     3.  ]
[4. 4. 4. ... 4. 4. 4.]
[5. 5. 5. ... 5. 5. 5.]
[6. 6. 6. ... 6. 6. 6.]
[7. 7. 7. ... 7. 7. 7.]
[8. 8. 8. ... 8. 8. 8.]
[9. 9. 9. ... 9. 9. 9.]
[10. 10. 10. ... 10. 10. 10.]
[11. 11. 11. ... 11. 11. 11.]
[12. 12. 12. ... 12. 12. 12.]
[13. 13. 13. ... 13. 13. 13.]
[14. 14. 14. ... 14. 14. 14.]
[15. 15. 15. ... 15. 15. 15.]
[16. 16. 16. ... 16. 16. 16.]
[17. 17. 17. ... 17. 17. 17.]
[18. 18. 18. ... 18. 18. 18.]
[19. 19. 19. ... 19. 19. 19.]
[20. 20. 20. ... 20. 20. 20.]
[21. 21. 21. ... 21. 21. 21.]
[22. 22. 22. ... 22. 22. 22.]
[23. 23. 23. ... 23. 23. 23.]
[24. 24. 24. ... 24. 24. 24.]
[25. 25. 25. ... 25. 25. 25.]
[26. 26. 26. ... 26. 26. 26.]
[27. 27. 27. ... 27. 27. 27.]
[28. 28. 28. ... 28. 28. 28.]
[29. 29. 29. ... 29. 29. 29.]
[30. 30. 30. ... 30. 30. 30.]
[31. 31. 31. ... 31. 31. 31.

In [421]:
np_bin[:, 1419]

array([   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  , 2642.19])

In [404]:
cagr_1 = round(((np_bin[:,0][bin_size] / np_bin[:,0][0]) ** (1/t) - 1) * 100 , 3)

In [ ]:
Now we need to geralise this

In [387]:
(np_bin[:,0][bin_size] / np_bin[:,0][0]) ** (1/bin_size/12)

0.9999805239940587

In [405]:
cagr_1

-0.28

In [320]:
cagr['bin_0']

,bin_0
0,6.10
1,6.21
2,6.26
3,6.34
4,6.04
...,...
1415,0.00
1416,0.00
1417,0.00
1418,0.00


In [ ]:
df1["cagr"] = (((df1["price"]/df1["start price"])**(1/years_in_window))-1)*100
df1["Date"] = pd.to_datetime(df1["Date"])
df1.set_index('Date', inplace=True)


df1.head(185)


# In[51]:


plt.plot(df1["cagr"], marker="o", ms=1.5)

# Labelling 

plt.xlabel("Date")
plt.ylabel("CAGR")
plt.title("Historical CAGR")

#plt.xlim([datetime.date(1886, 1, 1), datetime.date(2018, 1, 1)])
plt.ylim(-20,20)

# Display

plt.show()
